# 🚀 Gerador Automático de Data Schemas
## CRED-CANCEL v3.0

Este notebook executa automaticamente:
- `DESCRIBE FORMATTED` para cada tabela
- `SELECT * FROM <tabela> LIMIT 10` para cada tabela
- Metadados adicionais (contagem, colunas, etc.)

### Tabelas processadas:
- **16 tabelas originais** (usr_sat_*)
- **13 tabelas intermediárias** (teste.*)

In [ ]:
# Imports
from pyspark.sql import SparkSession
import os
from datetime import datetime
from IPython.display import display, HTML, Markdown

In [ ]:
# Configurações
OUTPUT_DIR = "data_schemas"

# Criar diretórios
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/originais", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/intermediarias", exist_ok=True)

print(f"✅ Diretórios criados em: {OUTPUT_DIR}/")

In [ ]:
# Inicializar Spark Session
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

# Usar sessão existente ou criar nova
try:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)
    print(f"✅ Usando SparkSession existente (Spark {spark.version})")
except:
    spark = SparkSession.builder \
        .appName("DataSchema Generator") \
        .enableHiveSupport() \
        .getOrCreate()
    print(f"✅ Nova SparkSession criada (Spark {spark.version})")

## 📋 Definição de Tabelas

In [ ]:
# Definição de todas as tabelas
TABELAS_ORIGINAIS = {
    'usr_sat_ods': [
        'ods_decl_dime_raw',
        'vw_cad_contrib',
        'vw_ods_pagamento',
        'vw_sna_pgdasd_grupo_empresarial',
        'vw_ods_contrib',
        'vw_ods_dcip'
    ],
    'usr_sat_cadastro': [
        'ruc_protocolo',
        'ruc_general',
        'tab_sit_cad'
    ],
    'usr_sat_shared': [
        'tab_generica',
        'tab_munic'
    ],
    'usr_sat_auditoria': [
        'aud_empresa_sob_suspeita',
        'aud_empresa_suspeita'
    ]
}

TABELAS_INTERMEDIARIAS = {
    'teste': [
        'credito_dime',
        'credito_dime_completo',
        'credito_dime_textil',
        'credito_dime_metalmec',
        'credito_dime_tech',
        'cancel_cnpj',
        'cancel_cadastro',
        'cancel_recolhimento',
        'cancel_suspeitas',
        'cancel_suspeitas_score',
        'cancel_zero_normal',
        'cancel_zero_simples',
        'cancel_final'
    ]
}

# Contar tabelas
total_originais = sum(len(v) for v in TABELAS_ORIGINAIS.values())
total_intermediarias = sum(len(v) for v in TABELAS_INTERMEDIARIAS.values())
total_geral = total_originais + total_intermediarias

print(f"📊 Total de tabelas a processar: {total_geral}")
print(f"   - Originais: {total_originais}")
print(f"   - Intermediárias: {total_intermediarias}")

## 🛠️ Funções Auxiliares

In [ ]:
def salvar_resultado(conteudo, tipo, database, tabela):
    """Salva resultado em arquivo"""
    categoria = "originais" if database != "teste" else "intermediarias"
    filepath = f"{OUTPUT_DIR}/{categoria}/{database}.{tabela}_{tipo}.txt"
    
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(conteudo)
    
    return filepath


def formatar_describe(df):
    """Formata output do DESCRIBE FORMATTED"""
    linhas = []
    linhas.append("=" * 100)
    linhas.append("DESCRIBE FORMATTED")
    linhas.append("=" * 100)
    linhas.append("")
    
    for row in df.collect():
        col_name = str(row[0]) if row[0] else ""
        data_type = str(row[1]) if row[1] else ""
        comment = str(row[2]) if len(row) > 2 and row[2] else ""
        
        linhas.append(f"{col_name:<35} {data_type:<25} {comment}")
    
    return "\n".join(linhas)


def formatar_select(df, tabela_nome):
    """Formata output do SELECT * LIMIT 10"""
    linhas = []
    linhas.append("=" * 100)
    linhas.append(f"SELECT * FROM {tabela_nome} LIMIT 10")
    linhas.append("=" * 100)
    linhas.append("")
    
    # Cabeçalho
    colunas = df.columns
    linhas.append(" | ".join(colunas))
    linhas.append("-" * 100)
    
    # Dados
    rows = df.collect()
    if len(rows) == 0:
        linhas.append("(Nenhum registro encontrado)")
    else:
        for row in rows:
            valores = [str(val)[:50] if val is not None else "NULL" for val in row]
            linhas.append(" | ".join(valores))
    
    linhas.append("")
    linhas.append(f"Total de registros exibidos: {len(rows)}")
    
    return "\n".join(linhas)


print("✅ Funções auxiliares definidas")

## 🔄 Função de Processamento Principal

In [ ]:
def processar_tabela(spark, database, tabela):
    """Processa uma tabela individual"""
    tabela_completa = f"{database}.{tabela}"
    
    print(f"\n📊 Processando: {tabela_completa}")
    print("-" * 80)
    
    try:
        # 1. DESCRIBE FORMATTED
        print("  🔍 Executando DESCRIBE FORMATTED...")
        df_describe = spark.sql(f"DESCRIBE FORMATTED {tabela_completa}")
        conteudo_describe = formatar_describe(df_describe)
        arquivo_describe = salvar_resultado(conteudo_describe, "describe", database, tabela)
        print(f"  ✅ DESCRIBE salvo: {arquivo_describe}")
        
        # 2. SELECT * LIMIT 10
        print("  🔍 Executando SELECT * LIMIT 10...")
        df_select = spark.sql(f"SELECT * FROM {tabela_completa} LIMIT 10")
        conteudo_select = formatar_select(df_select, tabela_completa)
        arquivo_select = salvar_resultado(conteudo_select, "select", database, tabela)
        print(f"  ✅ SELECT salvo: {arquivo_select}")
        
        # 3. Metadados adicionais
        print("  📈 Coletando metadados...")
        count = spark.sql(f"SELECT COUNT(*) as total FROM {tabela_completa}").collect()[0].total
        
        metadata = []
        metadata.append("=" * 100)
        metadata.append("METADADOS")
        metadata.append("=" * 100)
        metadata.append(f"Tabela: {tabela_completa}")
        metadata.append(f"Total de registros: {count:,}")
        metadata.append(f"Total de colunas: {len(df_select.columns)}")
        metadata.append(f"Data da extração: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        metadata.append("")
        metadata.append("Colunas:")
        for col in df_select.columns:
            metadata.append(f"  - {col}")
        
        arquivo_metadata = salvar_resultado("\n".join(metadata), "metadata", database, tabela)
        print(f"  ✅ METADATA salvo: {arquivo_metadata}")
        
        print(f"  ✅ ✅ ✅ Concluído: {tabela_completa} ({count:,} registros)")
        
        return True, count
        
    except Exception as e:
        print(f"  ❌ ERRO em {tabela_completa}: {str(e)}")
        
        # Salvar log de erro
        erro_msg = f"ERRO ao processar {tabela_completa}\n"
        erro_msg += f"Timestamp: {datetime.now()}\n"
        erro_msg += f"Erro: {str(e)}\n"
        salvar_resultado(erro_msg, "ERRO", database, tabela)
        
        return False, 0


print("✅ Função de processamento definida")

## 🎯 PRIORIDADE ALTA - Tabelas do Streamlit

Processando as 4 tabelas principais usadas no dashboard Streamlit:

In [ ]:
# Tabelas de PRIORIDADE ALTA (usadas no Streamlit)
PRIORIDADE_ALTA = [
    ('teste', 'credito_dime_completo'),
    ('teste', 'credito_dime_textil'),
    ('teste', 'credito_dime_metalmec'),
    ('teste', 'credito_dime_tech')
]

print("=" * 80)
print("🎯 PROCESSANDO TABELAS DE PRIORIDADE ALTA")
print("=" * 80)
print()

resultados_alta = []
for database, tabela in PRIORIDADE_ALTA:
    sucesso, count = processar_tabela(spark, database, tabela)
    resultados_alta.append((database, tabela, sucesso, count))

print("\n" + "=" * 80)
print("✅ PRIORIDADE ALTA CONCLUÍDA")
print("=" * 80)
for db, tab, suc, cnt in resultados_alta:
    status = "✅" if suc else "❌"
    print(f"{status} {db}.{tab}: {cnt:,} registros")

## 📁 TABELAS ORIGINAIS (usr_sat_*)

Processando todas as tabelas fonte do banco de produção:

In [ ]:
print("=" * 80)
print("📁 PROCESSANDO TABELAS ORIGINAIS")
print("=" * 80)
print()

resultados_originais = []

for database, tabelas in TABELAS_ORIGINAIS.items():
    print(f"\n🗂️  Database: {database}")
    print("=" * 80)
    
    for tabela in tabelas:
        sucesso, count = processar_tabela(spark, database, tabela)
        resultados_originais.append((database, tabela, sucesso, count))

print("\n" + "=" * 80)
print("✅ TABELAS ORIGINAIS CONCLUÍDAS")
print("=" * 80)

## 🔄 TABELAS INTERMEDIÁRIAS (teste.*)

Processando todas as tabelas intermediárias criadas no processamento:

In [ ]:
print("=" * 80)
print("🔄 PROCESSANDO TABELAS INTERMEDIÁRIAS")
print("=" * 80)
print()

resultados_intermediarias = []

for database, tabelas in TABELAS_INTERMEDIARIAS.items():
    print(f"\n🗂️  Database: {database}")
    print("=" * 80)
    
    for tabela in tabelas:
        # Pular se já foi processada na prioridade alta
        if (database, tabela) in PRIORIDADE_ALTA:
            print(f"⏭️  Pulando {database}.{tabela} (já processada em PRIORIDADE ALTA)")
            continue
        
        sucesso, count = processar_tabela(spark, database, tabela)
        resultados_intermediarias.append((database, tabela, sucesso, count))

print("\n" + "=" * 80)
print("✅ TABELAS INTERMEDIÁRIAS CONCLUÍDAS")
print("=" * 80)

## 📊 RELATÓRIO FINAL

In [ ]:
# Consolidar resultados
todos_resultados = resultados_alta + resultados_originais + resultados_intermediarias

total_processadas = len(todos_resultados)
total_sucesso = sum(1 for r in todos_resultados if r[2])
total_falhas = total_processadas - total_sucesso
total_registros = sum(r[3] for r in todos_resultados if r[2])

print("=" * 80)
print("📊 RELATÓRIO FINAL - GERAÇÃO DE DATA SCHEMAS")
print("=" * 80)
print()
print(f"📅 Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()
print(f"📈 Total de tabelas processadas: {total_processadas}")
print(f"   ✅ Sucesso: {total_sucesso}")
print(f"   ❌ Falhas: {total_falhas}")
print()
print(f"💾 Total de registros nas tabelas: {total_registros:,}")
print()
print(f"📁 Arquivos salvos em: {os.path.abspath(OUTPUT_DIR)}/")
print()

# Detalhamento por categoria
print("=" * 80)
print("📋 DETALHAMENTO POR CATEGORIA")
print("=" * 80)
print()

print("🎯 PRIORIDADE ALTA (Streamlit):")
for db, tab, suc, cnt in resultados_alta:
    status = "✅" if suc else "❌"
    print(f"  {status} {db}.{tab}: {cnt:,} registros")

print()
print("📁 TABELAS ORIGINAIS:")
for db, tab, suc, cnt in resultados_originais:
    status = "✅" if suc else "❌"
    print(f"  {status} {db}.{tab}: {cnt:,} registros")

print()
print("🔄 TABELAS INTERMEDIÁRIAS:")
for db, tab, suc, cnt in resultados_intermediarias:
    status = "✅" if suc else "❌"
    print(f"  {status} {db}.{tab}: {cnt:,} registros")

print()
print("=" * 80)
print("✅ PROCESSO CONCLUÍDO COM SUCESSO!")
print("=" * 80)

## 📋 Gerar Arquivo Índice

In [ ]:
# Gerar índice completo
indice = []
indice.append("=" * 100)
indice.append("ÍNDICE DE DATA SCHEMAS - CRED-CANCEL v3.0")
indice.append("=" * 100)
indice.append(f"Data de geração: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
indice.append(f"Total de tabelas: {total_processadas}")
indice.append(f"Processadas com sucesso: {total_sucesso}")
indice.append(f"Falhas: {total_falhas}")
indice.append(f"Total de registros: {total_registros:,}")
indice.append("")
indice.append("=" * 100)
indice.append("🎯 TABELAS DE PRIORIDADE ALTA (Streamlit)")
indice.append("=" * 100)
for db, tab, suc, cnt in resultados_alta:
    status = "✅" if suc else "❌"
    indice.append(f"{status} {db}.{tab} - {cnt:,} registros")

indice.append("")
indice.append("=" * 100)
indice.append("📁 TABELAS ORIGINAIS")
indice.append("=" * 100)
for db, tab, suc, cnt in resultados_originais:
    status = "✅" if suc else "❌"
    indice.append(f"{status} {db}.{tab} - {cnt:,} registros")

indice.append("")
indice.append("=" * 100)
indice.append("🔄 TABELAS INTERMEDIÁRIAS")
indice.append("=" * 100)
for db, tab, suc, cnt in resultados_intermediarias:
    status = "✅" if suc else "❌"
    indice.append(f"{status} {db}.{tab} - {cnt:,} registros")

indice.append("")
indice.append("=" * 100)
indice.append("📁 ESTRUTURA DE ARQUIVOS")
indice.append("=" * 100)
indice.append("")
indice.append("Para cada tabela, foram gerados 3 arquivos:")
indice.append("  1. <database>.<tabela>_describe.txt  - Schema detalhado (DESCRIBE FORMATTED)")
indice.append("  2. <database>.<tabela>_select.txt    - Amostra de 10 registros (SELECT * LIMIT 10)")
indice.append("  3. <database>.<tabela>_metadata.txt  - Metadados (total registros, colunas, etc)")
indice.append("")
indice.append("Organização de diretórios:")
indice.append(f"  {OUTPUT_DIR}/")
indice.append("    ├── originais/      (tabelas usr_sat_*)")
indice.append("    ├── intermediarias/ (tabelas teste.*)")
indice.append("    └── INDEX.txt       (este arquivo)")

filepath = f"{OUTPUT_DIR}/INDEX.txt"
with open(filepath, 'w', encoding='utf-8') as f:
    f.write("\n".join(indice))

print(f"📋 Índice completo salvo em: {filepath}")
print("")
print("✅ Todos os data schemas foram gerados com sucesso!")

## 🎉 Conclusão

Os data schemas foram gerados com sucesso!

### 📂 Próximos passos:

1. Verifique os arquivos gerados em `data_schemas/`
2. Revise o arquivo `INDEX.txt` para um resumo completo
3. Use os arquivos `*_describe.txt` para documentação de schemas
4. Use os arquivos `*_select.txt` para exemplos de dados
5. Use os arquivos `*_metadata.txt` para estatísticas das tabelas

### 📝 Formato dos arquivos:
- `<database>.<tabela>_describe.txt` → Schema completo (tipos, comentários)
- `<database>.<tabela>_select.txt` → 10 primeiras linhas de dados
- `<database>.<tabela>_metadata.txt` → Contagem, colunas, data de extração